Tutorial to analyze the StackOverflow developer survey dataset.

Dataset contains 64k+ responses to 60 questions(although many questions are optional). 

In [2]:
# EDA from jovian.ai

import pandas as pd

In [16]:
survey_raw_df = pd.read_csv('/Users/abeygailpanganiban/Desktop/developer_survey_2020/survey_results_public20.csv')

#Load schema file as a Pandas Series with Column as the Index and QuestionText as the Value
schema_raw = pd.read_csv('/Users/abeygailpanganiban/Desktop/developer_survey_2020/survey_results_schema20.csv', index_col='Column').QuestionText

In [4]:
pd.set_option('display.max_columns',85)
pd.set_option('display.max_rows',85)

In [17]:
# now we can use schema_raw to retrieve full question text for any column in survey_raw_df
schema_raw

Column
Respondent                      Randomized respondent ID number (not in order ...
MainBranch                      Which of the following options best describes ...
Hobbyist                                                  Do you code as a hobby?
Age                             What is your age (in years)? If you prefer not...
Age1stCode                      At what age did you write your first line of c...
CompFreq                         Is that compensation weekly, monthly, or yearly?
CompTotal                       What is your current total compensation (salar...
ConvertedComp                   Salary converted to annual USD salaries using ...
Country                                                        Where do you live?
CurrencyDesc                    Which currency do you use day-to-day? If your ...
CurrencySymbol                  Which currency do you use day-to-day? If your ...
DatabaseDesireNextYear          Which database environments have you done exte...
DatabaseW

In [5]:
survey_raw_df.head()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,CurrencySymbol,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,Ethnicity,Gender,JobFactors,JobSat,JobSeek,LanguageDesireNextYear,LanguageWorkedWith,MiscTechDesireNextYear,MiscTechWorkedWith,NEWCollabToolsDesireNextYear,NEWCollabToolsWorkedWith,NEWDevOps,NEWDevOpsImpt,NEWEdImpt,NEWJobHunt,NEWJobHuntResearch,NEWLearn,NEWOffTopic,NEWOnboardGood,NEWOtherComms,NEWOvertime,NEWPurchaseResearch,NEWPurpleLink,NEWSOSites,NEWStuck,OpSys,OrgSize,PlatformDesireNextYear,PlatformWorkedWith,PurchaseWhat,Sexuality,SOAccount,SOComm,SOPartFreq,SOVisitFreq,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,EUR,Microsoft SQL Server,Elasticsearch;Microsoft SQL Server;Oracle,"Developer, desktop or enterprise applications;...","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Independent contractor, freelancer, or self-em...",White or of European descent,Man,"Languages, frameworks, and other technologies ...",Slightly satisfied,I am not interested in new job opportunities,C#;HTML/CSS;JavaScript,C#;HTML/CSS;JavaScript,.NET Core;Xamarin,.NET;.NET Core,Microsoft Teams;Microsoft Azure;Trello,Confluence;Jira;Slack;Microsoft Azure;Trello,No,Somewhat important,Fairly important,NaN,NaN,Once a year,Not sure,NaN,No,Often: 1-2 days per week or more,Start a free trial;Ask developers I know/work ...,Amused,Stack Overflow (public Q&A for anyone who codes),Visit Stack Overflow;Go for a walk or other ph...,Windows,2 to 9 employees,Android;iOS;Kubernetes;Microsoft Azure;Windows,Windows,NaN,Straight / Heterosexual,No,"No, not at all",NaN,Multiple times per day,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,GBP,NaN,NaN,"Developer, full-stack;Developer, mobile","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed full-time,NaN,NaN,NaN,Very dissatisfied,I am not interested in new job opportunities,Python;Swift,JavaScript;Swift,React Native;TensorFlow;Unity 3D,React Native,Github;Slack,Confluence;Jira;Github;Gitlab;Slack,NaN,NaN,Fairly important,NaN,NaN,Once a year,Not sure,NaN,No,NaN,NaN,Amused,Stack Overflow (public Q&A for anyone who code...,Visit Stack Overflow;Go for a walk or other ph...,MacOS,"1,000 to 4,999 employees",iOS;Kubernetes;Linux;MacOS,iOS,I have little or no influence,NaN,Yes,"Yes, definitely",Less than once per month or monthly,Multiple times per day,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Objective-C;Python;Swift,Objective-C;Python;Swift,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Once a decade,NaN,NaN,No,NaN,NaN,NaN,Stack Overflow (public Q&A for anyone who codes),NaN,Linux-based,NaN,NaN,NaN,NaN,NaN,Yes,"Yes, somewhat",A few times per month or weekly,Daily or almost daily,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,ALL,NaN,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",NaN,White or of European descent,Man,Flex time or a flexible schedule;Office enviro...,Slightly dissatisfied,"I’m not actively looking, but I am open to new...",NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,Not at all important/not necessary,Curious about other opportunities;Wanting to w...,NaN,Once a year,Not sure,Yes,Yes,Occasionally: 1-2 days per quarter but less th...,NaN,NaN,Stack Overflow (public Q&A for anyone who code...,NaN,Lin

In [6]:
#Review list of columns in the dataframe

survey_raw_df.columns

Index([u'Respondent', u'MainBranch', u'Hobbyist', u'Age', u'Age1stCode',
       u'CompFreq', u'CompTotal', u'ConvertedComp', u'Country',
       u'CurrencyDesc', u'CurrencySymbol', u'DatabaseDesireNextYear',
       u'DatabaseWorkedWith', u'DevType', u'EdLevel', u'Employment',
       u'Ethnicity', u'Gender', u'JobFactors', u'JobSat', u'JobSeek',
       u'LanguageDesireNextYear', u'LanguageWorkedWith',
       u'MiscTechDesireNextYear', u'MiscTechWorkedWith',
       u'NEWCollabToolsDesireNextYear', u'NEWCollabToolsWorkedWith',
       u'NEWDevOps', u'NEWDevOpsImpt', u'NEWEdImpt', u'NEWJobHunt',
       u'NEWJobHuntResearch', u'NEWLearn', u'NEWOffTopic', u'NEWOnboardGood',
       u'NEWOtherComms', u'NEWOvertime', u'NEWPurchaseResearch',
       u'NEWPurpleLink', u'NEWSOSites', u'NEWStuck', u'OpSys', u'OrgSize',
       u'PlatformDesireNextYear', u'PlatformWorkedWith', u'PurchaseWhat',
       u'Sexuality', u'SOAccount', u'SOComm', u'SOPartFreq', u'SOVisitFreq',
       u'SurveyEase', u'SurveyLeng

## Data Prep & Cleaning

Limit analysis to following areas:
* Demographics of survey respondents and global programming community
* distribution of programming skills, exp, preferences
* employment-related info, preferences, and opinions

Select subset of columns

In [7]:
selected_columns = [
    # Demographics
    'Country',
    'Age',
    'Gender',
    'EdLevel',
    'UndergradMajor',
    # Programming experience
    'Hobbyist',
    'Age1stCode',
    'YearsCode',
    'YearsCodePro',
    'LanguageWorkedWith',
    'LanguageDesireNextYear',
    'NEWLearn',
    'NEWStuck',
    # Employment
    'Employment',
    'DevType',
    'WorkWeekHrs',
    'JobSat',
    'JobFactors',
    'NEWOvertime',
    'NEWEdImpt'
]

In [8]:
len(selected_columns)

20

In [9]:
survey_df = survey_raw_df[selected_columns].copy()

In [18]:
schema = schema_raw[selected_columns]

In [19]:
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64461 entries, 0 to 64460
Data columns (total 20 columns):
Country                   64072 non-null object
Age                       45446 non-null float64
Gender                    50557 non-null object
EdLevel                   57431 non-null object
UndergradMajor            50995 non-null object
Hobbyist                  64416 non-null object
Age1stCode                57900 non-null object
YearsCode                 57684 non-null object
YearsCodePro              46349 non-null object
LanguageWorkedWith        57378 non-null object
LanguageDesireNextYear    54113 non-null object
NEWLearn                  56156 non-null object
NEWStuck                  54983 non-null object
Employment                63854 non-null object
DevType                   49370 non-null object
WorkWeekHrs               41151 non-null float64
JobSat                    45194 non-null object
JobFactors                49349 non-null object
NEWOvertime              

Most columns have data type object, either because they contain values of different types or contain empty values (NaN). It appears that every column contains some empty values since Non-Null count for every column is lower than the total number of rows(64461). We'll deal with empty values and manuually adjust data type for each column on a case by case basis.

only two columns detected as numeric. 

Let's convert some other oclumns into numeric data types while ignoring non-numeric values. 

In [20]:
survey_df['Age1stCode'] = pd.to_numeric(survey_df.Age1stCode, errors='coerce')
survey_df['YearsCode'] = pd.to_numeric(survey_df.YearsCode, errors='coerce')
survey_df['YearsCodePro'] = pd.to_numeric(survey_df.YearsCodePro, errors='coerce')

In [23]:
# View some basic statistics about numeric columns
survey_df.describe()

,Age,Age1stCode,YearsCode,YearsCodePro,WorkWeekHrs
count,45446.000000,57473.000000,56784.000000,44133.000000,41151.000000
mean,30.834111,15.476572,12.782051,8.869667,40.782174
std,9.585392,5.114081,9.490657,7.759961,17.816383
min,1.000000,5.000000,1.000000,1.000000,1.000000
25%,24.000000,12.000000,6.000000,3.000000,40.000000
50%,29.000000,15.000000,10.000000,6.000000,40.000000
75%,35.000000,18.000000,17.000000,12.000000,44.000000
max,279.000000,85.000000,50.000000,50.000000,475.000000


Problem with age column:
* min = 1, max = 100+. Common issue with surveys: responses may contain invalid values due to accidental or intentional errors while responding

Simple Fix: 
* ignore rows where age is higher than 100 years and lower than 10 years

In [24]:
survey_df.drop(survey_df[survey_df.Age < 10].index, inplace=True)
survey_df.drop(survey_df[survey_df.Age > 100].index, inplace=True)

Same problem with WorkWeekHrs. 
* Ignore entries where value is higher than 140 hours (~20 hrs/day)

In [25]:
survey_df.drop(survey_df[survey_df.WorkWeekHrs > 140].index, inplace=True)

Gender Column:
* column allows for picking multiple optinos. We'll remove values containing more than one option to simplify analysis.

In [26]:
survey_df['Gender'].value_counts()

Man                                                            45895
Woman                                                           3835
Non-binary, genderqueer, or gender non-conforming                385
Man;Non-binary, genderqueer, or gender non-conforming            121
Woman;Non-binary, genderqueer, or gender non-conforming           92
Woman;Man                                                         73
Woman;Man;Non-binary, genderqueer, or gender non-conforming       25
Name: Gender, dtype: int64

In [27]:
import numpy as np

In [28]:
survey_df.where(~(survey_df.Gender.str.contains(';',na=False)), np.nan, inplace=True)

Dataset prepared for analysis. Look at sample of rows from dataframe

In [29]:
survey_df.sample(10)

,Country,Age,Gender,EdLevel,UndergradMajor,Hobbyist,Age1stCode,YearsCode,YearsCodePro,LanguageWorkedWith,LanguageDesireNextYear,NEWLearn,NEWStuck,Employment,DevType,WorkWeekHrs,JobSat,JobFactors,NEWOvertime,NEWEdImpt
17304,Brazil,18.0,Man,Some college/university study without earning ...,"Information systems, information technology, o...",Yes,15.0,3.0,NaN,HTML/CSS;JavaScript;TypeScript,Dart;HTML/CSS;JavaScript;TypeScript,Every few months,Meditate;Visit Stack Overflow;Watch help / tut...,Employed full-time,"Developer, back-end;Developer, front-end;Devel...",80.0,Very dissatisfied,NaN,Occasionally: 1-2 days per quarter but less th...,Very important
48810,United States,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Mathematics or statistics,Yes,8.0,13.0,NaN,Bash/Shell/PowerShell;C;C++;Go;Haskell;HTML/CS...,Haskell,Every few months,Meditate;Play games;Call a coworker or friend;...,Student,NaN,NaN,NaN,NaN,NaN,NaN
27631,Germany,NaN,Man,NaN,"Computer science, computer engineering, or sof...",Yes,10.0,5.0,NaN,C;C++;HTML/CSS;Java;JavaScript,C;C++;HTML/CSS,Once every few years,Play games;Visit Stack Overflow,Student,NaN,NaN,NaN,Flex time or a flexible schedule;Financial per...,NaN,NaN
35824,Iran,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Computer science, computer engineering, or sof...",No,19.0,10.0,5.0,C#;JavaScript;PHP;SQL,Dart;Python;Swift,Every few months,Visit Stack Overflow;Go for a walk or other ph...,Employed part-time,Academic researcher;Database administrator;Des...,70.0,Slightly satisfied,NaN,Often: 1-2 days per week or more,Fairly important
54052,United Arab Emirates,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Information systems, information technology, o...",Yes,19.0,6.0,2.0,HTML/CSS;Java;JavaScript;Kotlin,NaN,Every few months,Visit Stack Overflow;Visit another developer c...,Employed full-time,"Developer, mobile",63.0,Neither satisfied nor dissatisfied,NaN,Sometimes: 1-2 days per month but less than we...,Critically important
25268,United States,28.0,Man,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",Yes,12.0,16.0,8.0,Bash/Shell/PowerShell;C#;HTML/CSS;JavaScript;K...,Bash/Shell/PowerShell;C#;JavaScript;Kotlin;Pyt...,Every few months,Meditate;Call a coworker or friend;Visit Stack...,Employed full-time,"Database administrator;Developer, back-end;Dev...",60.0,Very satisfied,"Languages, frameworks, and other technologies ...",Often: 1-2 days per week or more,Not at all important/not necessary
19891,United States,NaN,Man,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Another engineering discipline (such as civil,...",No,19.0,16.0,14.0,Java;Python,Kotlin,Every few months,Visit Stack Overflow,Employed full-time,"Developer, back-end",NaN,Slightly satisfied,NaN,Sometimes: 1-2 days per month but less than we...,NaN
47604,New Zealand,NaN,Woman,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Mathematics or statistics,Yes,8.0,20.0,11.0,Python;R;SQL;VBA,C#;JavaScript;Python;R;SQL,Once a year,Call a coworker or friend;Visit Stack Overflow...,Employed full-time,Database administrator,NaN,Very satisfied,"Languages, frameworks, and other technologies ...",Occasionally: 1-2 days per quarter but less th...,Fairly important
18633,United States,NaN,Man,"Secondary school (e.g. American high school, G...",NaN,Yes,14.0,5.0,NaN,C;HTML/CSS;Java,Dart;JavaScript,Once a year,Meditate;Visit Stack Overflow;Watch help / tut...,Student,NaN,NaN,NaN,Industry that I’d be working in;Remote work op...,NaN,NaN
57620,Nigeria,49.0,Man,NaN,"Another engineering discipline (such as civil,...",No,45.0,3.0,NaN,C#;HTML/CSS;Java;JavaScript;PHP;SQL,Assembly,Every few months,Call a coworker or friend;Visit Stack Overflow...,"Independent contractor, freelancer, or self-em...","Developer, back-end;Developer, embedded applic...",8.0,Slightly satisfied,Flex time or a flexible schedule;Financial per...,Often: 1-2 days per week or more,Fairly important
